We define $B_1 = \alpha \left(\frac{m}{1+\alpha}\right)^{\frac{1+\alpha}{\alpha}}$, $B_1^{dif} = \frac{1+\alpha}{\alpha} B_1$ and $B_2 = \left(\frac{m}{1+\alpha}\right)^{\frac{1}{\alpha}}$.

# 1) Surpluses in a world with unconditional policy
We need to solve
$$r\Sigma_a\left(b,\lambda\right) = \bar{w}-\max\{b,b_1\}-q_a \left[ \chi\Sigma_a \left(b_{UI},\lambda_1\right)+(1-\chi)\Sigma_a \left(b_1,0\right)\right]-B_1\Sigma_a\left(b,\lambda\right)^{\frac{1+\alpha}{\alpha}}+\lambda \left[\Sigma_a\left( b_1,0\right)- \Sigma_a\left(b,\lambda\right)\right]+\dot{\Sigma}_a\left(b,\lambda\right)$$
for $(b,\lambda)$ equal to $(b_1,0)$, $(b_{UI},\lambda_0)$ and $(b_{UI},\lambda_1)$.

We call accordingly
$$
\begin{align*}
    & \Sigma_1 = \Sigma_a(b_1,0)\\
    & \Sigma_2 = \Sigma_a(b_{UI},\lambda_0)\\
    & \Sigma_3 =\Sigma_a(b_{UI},\lambda_1)
\end{align*}
$$



We solve simultaneously for the system
$$
\begin{align*}
    & r\Sigma_1 = \bar{w}-b_1-q \left[\chi \Sigma_3+(1-\chi)\Sigma_1\right] -B_1 \Sigma_1^{\frac{1+\alpha}{\alpha}}+\dot\Sigma_1\\
    & r\Sigma_2 = \bar{w}-b_{UI}-q \left[\chi \Sigma_3+(1-\chi)\Sigma_1\right]  -B_1 \Sigma_2^{\frac{1+\alpha}{\alpha}}+\lambda_0(\Sigma_1-\Sigma_2)+\dot\Sigma_2\\    
    & r\Sigma_3 = \bar{w}-b_{UI}-q \left[\chi \Sigma_3+(1-\chi)\Sigma_1\right]  -B_1 \Sigma_3^{\frac{1+\alpha}{\alpha}}+\lambda_1(\Sigma_1-\Sigma_3)+\dot\Sigma_3
\end{align*}
$$
We always set $\chi=1$ inthe first policy.
Note that $\Sigma_2=\Sigma_3$ with the first policy.

In [ ]:
## first policy
#=function Σ_obj_p1(Σ::Vector, C::Country1)
    Σ1 = ( 1 - C.b1 - C.q * Σ[2] - C.B1*Σ[1]^((1+C.α)/C.α) ) / C.r
    Σ2 = ( 1 - C.bUI - C.q * Σ[2] - C.B1*Σ[2]^((1+C.α)/C.α) + C.λ * (Σ[1]-Σ[2]) ) / C.r
    out = (Σ1-Σ[1])^2 + (Σ2-Σ[2])^2
end=#


function Σ_obj_p1(Σ::Vector, grad::Vector, C::Country1)
    d1 = 1 - C.b1 - C.q * Σ[2] - C.B1*Σ[1]^((1+C.α)/C.α) - C.r * Σ[1]
    d2 = 1 - C.bUI - C.q * Σ[2] - C.B1*Σ[2]^((1+C.α)/C.α) + C.λ * (Σ[1]-Σ[2]) - C.r * Σ[2]
    grad[1] = 2*d1* (- C.B1dif * Σ[1]^(1/C.α)-C.r ) + 2*d2 * C.λ 
    grad[2] = 2*d1* (-C.q) + 2*d2 * (-C.q - C.B1dif * Σ[2]^(1/C.α)-C.λ-C.r )
    return d1^2 + d2^2
end

function find_Σ(C::Country1)
    opt = Opt(:LD_MMA, 2)
    lower_bounds!( opt, zeros(2) )
    xtol_rel!(opt,1e-8)
    min_objective!( opt, (v,g)-> Σ_obj_p1(v,g,C) )
    (minf,Σ,ret) = optimize( opt, zeros(2) )
    return Σ
end


## second policy
#=function Σ_obj_p2(Σ::Vector, C::Country2)
    Σ1 = ( 1 - C.b0 - C.q * Σ[3] - C.B1*Σ[1]^((1+C.α)/C.α) ) / C.r
    Σ2 = ( 1 - C.bUI - C.q * Σ[3] - C.B1*Σ[2]^((1+C.α)/C.α) + C.λmax * (Σ[1]-Σ[2]) ) / C.r
    Σ3 = ( 1 - C.bUI - C.q * Σ[3] - C.B1*Σ[3]^((1+C.α)/C.α) + C.λmin * (Σ[1]-Σ[3]) )/ C.r
    out = (Σ1-Σ[1])^2 + (Σ2-Σ[2])^2 + (Σ3-Σ[3])^2
end=#


function Σ_obj_p2(Σ::Vector, grad::Vector, C::Country2)
    d1 = 1 - C.b0 - C.q * Σ[3] - C.B1*Σ[1]^((1+C.α)/C.α) - C.r * Σ[1]
    d2 = 1 - C.bUI - C.q * Σ[3] - C.B1*Σ[2]^((1+C.α)/C.α) + C.λ0 * (Σ[1]-Σ[2]) - C.r * Σ[2]
    d3 = 1 - C.bUI - C.q * Σ[3] - C.B1*Σ[3]^((1+C.α)/C.α) + C.λ1 * (Σ[1]-Σ[3]) - C.r * Σ[3]
    grad[1] = 2*d1* (- C.B1dif * Σ[1]^(1/C.α)-C.r ) + 2*d2 * C.λ0 +  2*d3 * C.λ1
    grad[2] = 2*d2 * (- C.B1dif * Σ[2]^(1/C.α)-C.λ0-C.r )
    grad[3] = 2*d1* (-C.q) + 2*d2 * (-C.q) +  2*d3 * (-C.q - C.B1dif * Σ[3]^(1/C.α) - C.λ1 - C.r)
    return d1^2 + d2^2 + d3^2
end

function find_Σ(C::Country2)
    opt = Opt(:LD_MMA, 3)
    lower_bounds!( opt, zeros(3) )
    xtol_rel!(opt,1e-8)
    min_objective!( opt, (v,g)-> Σ_obj_p2(v,g,C) )
    (minf,Σ,ret) = optimize( opt, zeros(3) )
    return Σ
end

For the life-cycle version, we define $\Sigma^{con}_i(A_{ret}-a)=\Sigma_i(a)$ for $a\leq A_{ret}$. Note $\Sigma_{ia}(a)=-\Sigma^{con}_{ia}(A-a)$. We solve a traditional system of differential equations with initial conditions $\Sigma^{con}_{ia}(0)=0$.

In [ ]:
function ODE_Σcon!(dΣcon::Vector, Σcon::Vector, t::Float64, C::Country1A)
    q = qaux(C.Aret - t, C) ## t = C.Aret - a
    dΣcon[1] = 1 - C.b1 - q * Σcon[2] - C.B1*Σcon[1]^((1+C.α)/C.α) - C.r * Σcon[1]
    dΣcon[2] = 1 - C.bUI - q * Σcon[2] - C.B1*Σcon[2]^((1+C.α)/C.α) + C.λ * (Σcon[1]-Σcon[2]) - C.r * Σcon[2]
end

function ODE_Σcon!(dΣcon::Vector, Σcon::Vector, t::Float64, C::Country2A)
    q = qaux(C.Aret - t, C) ## t = C.Aret - a
    dΣcon[1] = 1 - C.b0 - q * (C.χ * Σcon[3] + (1-C.χ) * Σcon[1]) - C.B1*Σcon[1]^((1+C.α)/C.α) - C.r * Σcon[1]
    dΣcon[2] = 1 - C.bUI - q * (C.χ * Σcon[3] + (1-C.χ) * Σcon[1]) - C.B1*Σcon[2]^((1+C.α)/C.α) + C.λ0 * (Σcon[1]-Σcon[2]) - C.r * Σcon[2]
    dΣcon[3] = 1 - C.bUI - q * (C.χ * Σcon[3] + (1-C.χ) * Σcon[1]) - C.B1*Σcon[3]^((1+C.α)/C.α) + C.λ1 * (Σcon[1]-Σcon[3]) - C.r * Σcon[3]
end

function find_Σ(C::Country1A; reltol = 1e-8)
    Σcon0 = zeros(2) ## here is the difference with below
    tspan = (0., C.Aret-C.Amin)
    prob = ODEProblem((dΣcon,Σcon,p,t)->ODE_Σcon!(dΣcon,Σcon,t,C),Σcon0,tspan)
    sol = solve(prob, reltol=reltol) ##sol is Σcon
    return a -> sol(C.Aret-a)
end

function find_Σ(C::Country2A; reltol = 1e-8)
    Σcon0 = zeros(3)
    tspan = (0., C.Aret-C.Amin)
    prob = ODEProblem((dΣcon,Σcon,p,t)->ODE_Σcon!(dΣcon,Σcon,t,C),Σcon0,tspan)
    sol = solve(prob, reltol=reltol) ##sol is Σcon
    return a -> sol(C.Aret-a)
end

# 2) Surpluses in a world without the policy
We need to solve
$$r\Psi_a\left(b,\lambda\right) = \bar{w}-\max\{b,b_0\}-q_a \left[ \chi\Psi_a \left(b_{UI},\lambda_0\right)+(1-\chi)\Psi_a \left(b_0,0\right)\right]-B_1\Psi_a\left(b,\lambda\right)^{\frac{1+\alpha}{\alpha}}+\lambda \left[\Psi_a\left( b_0,0\right)- \Psi_a\left(b,\lambda\right)\right]+\dot{\Psi}_a\left(b,\lambda\right)$$
for $(b,\lambda)$ equal to $(b_0,0)$ and $(b_{UI},\lambda_0)$.

We solve
$$
\begin{align*}
    & r\Psi_1 = \bar{w}-b_{0}-q \left[\chi\Psi_2+(1-\chi)\Psi_1\right] -B_1 \Psi_1^{\frac{1+\alpha}{\alpha}}+\dot\Psi_1\\
    & r\Psi_2 = \bar{w}-b_{UI}-q \left[\chi\Psi_2+(1-\chi)\Psi_1\right] -B_1 \Psi_2^{\frac{1+\alpha}{\alpha}}+\lambda_0(\Psi_1-\Psi_2)+\dot\Psi_2
\end{align*}
$$

In [ ]:
## first policy
#=function Y_obj_p1(Y::Vector, C::Country1)
    Y1 = ( 1 - C.b0 - C.q * Y[2] - C.B1*Y[1]^((1+C.α)/C.α) ) / C.r
    Y2 = ( 1 - C.bUI - C.q * Y[2] - C.B1*Y[2]^((1+C.α)/C.α) + C.λ * (Y[1]-Y[2]) ) / C.r
    out = (Y1-Y[1])^2 + (Y2-Y[2])^2
end=#

function Ψ_obj_p1(Ψ::Vector, grad::Vector, C::Country1)
    d1 = 1 - C.b0 - C.q * Ψ[2] - C.B1*Ψ[1]^((1+C.α)/C.α) - C.r * Ψ[1]
    d2 = 1 - C.bUI - C.q * Ψ[2] - C.B1*Ψ[2]^((1+C.α)/C.α) + C.λ * (Ψ[1]-Ψ[2]) - C.r * Ψ[2]
    grad[1] = 2*d1* (- C.B1dif * Ψ[1]^(1/C.α)-C.r ) + 2*d2 * C.λ
    grad[2] = 2*d1* (-C.q) + 2*d2 * (-C.q - C.B1dif * Ψ[2]^(1/C.α)-C.λ-C.r )
    return d1^2 + d2^2
end

function find_Ψ(C::Country1)
    opt = Opt(:LD_MMA, 2)
    lower_bounds!( opt, zeros(2) )
    xtol_rel!(opt,1e-8)
    min_objective!( opt, (v,g)-> Ψ_obj_p1(v,g,C) )
    (minf,Ψ,ret) = optimize( opt, zeros(2) )
    return Ψ
end


## second policy
#=function Y_obj_p2(Y::Vector, C::Country2)
    Y1 = ( 1 - C.b0 - C.q * Y[2] - C.B1*Y[1]^((1+C.α)/C.α) ) / C.r
    Y2 = ( 1 - C.bUI - C.q * Y[2] - C.B1*Y[2]^((1+C.α)/C.α) + C.λ0 * (Y[1]-Y[2]) ) / C.r
    out = (Y1-Y[1])^2 + (Y2-Y[2])^2
end=#


function Ψ_obj_p2(Ψ::Vector, grad::Vector, C::Country2)
    d1 = 1 - C.b0 - C.q * Ψ[2] - C.B1*Ψ[1]^((1+C.α)/C.α) - C.r * Ψ[1]
    d2 = 1 - C.bUI - C.q * Ψ[2] - C.B1*Ψ[2]^((1+C.α)/C.α) + C.λ0 * (Ψ[1]-Ψ[2]) - C.r * Ψ[2]
    grad[1] = 2*d1* (- C.B1dif * Ψ[1]^(1/C.α)-C.r ) + 2*d2 * C.λ0
    grad[2] = 2*d1* (-C.q) + 2*d2 * (-C.q - C.B1dif * Ψ[2]^(1/C.α)-C.λ0-C.r )
    return d1^2 + d2^2
end

function find_Ψ(C::Country2)
    opt = Opt(:LD_MMA, 2)
    lower_bounds!( opt, zeros(2) )
    xtol_rel!(opt,1e-8)
    min_objective!( opt, (v,g)-> Ψ_obj_p2(v,g,C) )
    (minf,Ψ,ret) = optimize( opt, zeros(2) )
    return Ψ
end

For the life-cycle version, we define $\Psi^{con}_i(A_{ret}-a)=\Psi_i(a)$ for $a\leq A_{ret}$. Note $\Psi_{ia}(a)=-\Psi^{con}_{ia}(A-a)$. We solve a traditional system of differential equations with initial conditions $\Psi^{con}_{ia}(0)=0$.

In [ ]:
function ODE_Ψcon!(dΨcon::Vector, Ψcon::Vector, t::Float64, C::Country1A)
    q = qaux(C.Aret - t, C) ## t = C.Aret - a
    dΨcon[1] = 1 - C.b0 - q * Ψcon[2] - C.B1*Ψcon[1]^((1+C.α)/C.α) - C.r * Ψcon[1]
    dΨcon[2] = 1 - C.bUI - q * Ψcon[2] - C.B1*Ψcon[2]^((1+C.α)/C.α) + C.λ * (Ψcon[1]-Ψcon[2]) - C.r * Ψcon[2]
end

function ODE_Ψcon!(dΨcon::Vector, Ψcon::Vector, t::Float64, C::Country2A)
    q = qaux(C.Aret - t, C) ## t = C.Aret - a
    dΨcon[1] = 1 - C.b0 - q * (C.χ * Ψcon[2] + (1-C.χ) * Ψcon[1]) - C.B1*Ψcon[1]^((1+C.α)/C.α) - C.r * Ψcon[1]
    dΨcon[2] = 1 - C.bUI - q * (C.χ * Ψcon[2] + (1-C.χ) * Ψcon[1]) - C.B1*Ψcon[2]^((1+C.α)/C.α) + C.λ0 * (Ψcon[1]-Ψcon[2]) - C.r * Ψcon[2]
end

function find_Ψ(C::Union{Country1A,Country2A}; reltol = 1e-8)
    Ψcon0 = zeros(2)
    tspan = (0., C.Aret-C.Amin)
    prob = ODEProblem((dΨcon,Ψcon,p,t)->ODE_Ψcon!(dΨcon,Ψcon,t,C),Ψcon0,tspan)
    sol = solve(prob, reltol=reltol) ##sol is Σcon
    return a -> sol(C.Aret-a)
end

# 3) Surplus with age-conditional policy
We need to solve
$$rS_a\left(b,\lambda\right) = \bar{w}-\max\{b,b_{\mathbf{1}\left\{ a\geq A\right\}}\}-q_a \left[\chi S_a \left(b_{UI},  \lambda_{\mathbf{1} \left\{a\geq A\right\}}\right)+\left(1-\chi\right) S_a \left(b_{\mathbf{1} \left\{a\geq A\right\}},0\right)\right]-\mathcal R\left(S_a\left(b,\lambda\right)\right)
+\lambda \left[ S_a\left( b_{\mathbf{1}\left\{ a\geq A\right\}},0 \right)- S_a\left(b,\lambda\right)\right]+\dot{S}_a\left(b,\lambda\right)$$
for $(b,\lambda)$ equal to $(b_0,0)$, $(b_1,0)$, $(b_{UI},\lambda_0)$ and $(b_{UI},\lambda_1)$.

For $a<A$, we call
\begin{align*}
    & S_1 = S_a(b_0,0)\\
    & S_2 = S_a(b_{UI},\lambda_0)
\end{align*}
and for $a\geq A$, 
\begin{align*}
    & S_1 = S_a(b_1,0)\\
    & S_2 = S_a(b_{UI},\lambda_0)\\
    & S_3 = S_a(b_{UI},\lambda_1)
\end{align*}

Note that, for $a\geq A$, $S_i=\Sigma_i$ so we only solve for $a<A$.

We solve
$$
\begin{align*}
    & rS_1=\bar{w}-b_0-q\left[\chi S_2+(1-\chi)S_1\right]-B_1S_1^{\frac{1+\alpha}{\alpha}}+\dot S_1\\
    & rS_2=\bar{w}-b_{UI}-q\left[\chi S_2+(1-\chi)S_1\right]-B_1S_2^{\frac{1+\alpha}{\alpha}}+\lambda_0(S_1-S_2)+\dot S_{2}
\end{align*}
$$

We define $S^{con}_i(A-a)=S_i(a)$ for $a\leq A$. Note $S_{ia}(a)=-S^{con}_{ia}(A-a)$. We solve a traditional system of differential equations with initial conditions $S^{con}_1(0)=\Sigma_1(A)$ and $S^{con}_2(0)=\Sigma_2(A)$.

For $a\geq A$, we define $S_i(a)=\Sigma_i(a)$.

In [ ]:
## first policy
function ODE_Scon!(dScon::Vector, Scon::Vector, C::Country1)
    dScon[1] = 1 - C.b0 - C.q*Scon[2] - C.B1*Scon[1]^((1+C.α)/C.α) - C.r * Scon[1]
    dScon[2] = 1 - C.bUI - C.q*Scon[2] - C.B1*Scon[2]^((1+C.α)/C.α) + C.λ * (Scon[1]-Scon[2]) - C.r * Scon[2]
end

## second policy
function ODE_Scon!(dScon::Vector, Scon::Vector, C::Country2)
    dScon[1] = 1 - C.b0 - C.q*Scon[2] - C.B1*Scon[1]^((1+C.α)/C.α) - C.r * Scon[1]
    dScon[2] = 1 - C.bUI - C.q*Scon[2] - C.B1*Scon[2]^((1+C.α)/C.α) + C.λ0 * (Scon[1]-Scon[2]) - C.r * Scon[2]
end

function find_S(Scon0::Vector, C::Union{Country1,Country2}; reltol = 1e-8)
    tspan = (0., C.A-C.Amin)
    prob = ODEProblem((dScon,Scon,p,t)->ODE_Scon!(dScon,Scon,C),Scon0,tspan)
    sol = solve(prob, reltol=reltol) #alg_hints=[:stiff] if max(Scon, 0)
    return a -> sol(C.A-a)
end

For the first policy, we have the uncertainty parameter $\xi$ on top that affects the conditions when $a=A$, $S^{con}_1(0)=(1-\xi)\Sigma_1(A)+\xi \Psi_1(A)$ and $S^{con}_2(0)=(1-\xi)\Sigma_2(A)+\xi\Psi_2(A)$.

For the second policy, we have the UI recipiency rate.

In [ ]:
## first policy
function ODE_Scon!(dScon::Vector, Scon::Vector, t::Float64, C::Country1A)
    q = qaux(C.A - t, C) ## t = C.A - a
    dScon[1] = 1 - C.b0 - q*Scon[2] - C.B1*Scon[1]^((1+C.α)/C.α) - C.r * Scon[1]
    dScon[2] = 1 - C.bUI - q*Scon[2] - C.B1*Scon[2]^((1+C.α)/C.α) + C.λ * (Scon[1]-Scon[2]) - C.r * Scon[2]
end

## second policy
function ODE_Scon!(dScon::Vector, Scon::Vector, t::Float64, C::Country2A)
    q = qaux(C.A - t, C) ## t = C.A - a
    dScon[1] = 1 - C.b0 - q*(C.χ * Scon[2] + (1-C.χ) * Scon[1]) - C.B1*Scon[1]^((1+C.α)/C.α) - C.r * Scon[1]
    dScon[2] = 1 - C.bUI - q*(C.χ * Scon[2] + (1-C.χ) * Scon[1]) - C.B1*Scon[2]^((1+C.α)/C.α) + C.λ0 * (Scon[1]-Scon[2]) - C.r * Scon[2]
end

function find_S(Scon0::Vector, C::Union{Country1A,Country2A}; reltol = 1e-8)
    tspan = (0., C.A-C.Amin)
    prob = ODEProblem((dScon,Scon,p,t)->ODE_Scon!(dScon,Scon,t,C),Scon0,tspan)
    sol = solve(prob, reltol=reltol) #alg_hints=[:stiff] if max(Scon, 0)
    return a -> sol(C.A-a)
end

# 4) Wrap-up

In [ ]:
function find_Surplus(C::Union{Country1,Country2}; 
        reltol=1e-12, findΨ=true, findS=true)
    Σ = find_Σ(C)
    S = findS ? find_S(Σ[1:2], C, reltol=reltol) : zeros(2)
    Ψ = findΨ ? find_Ψ(C) : zeros(2)
    return Surplus(Σ=Σ, S=S, Ψ=Ψ)
end

function find_Surplus(C::Country1A; 
        reltol=1e-12, findΨ=true, findS=true)
    Σ = find_Σ(C)
    if C.ξ>0 
        Ψ = find_Ψ(C)
        Scon0 = (1-C.ξ) * Σ(C.A)[1:2] .+ C.ξ * Ψ(C.A)[1:2]
    else
        Ψ = findΨ ? find_Ψ(C) : zeros(2)
        Scon0 = Σ(C.A)[1:2]   
    end
    S = findS ? find_S(Scon0, C, reltol=reltol) : zeros(2)
    return Surplus(Σ=Σ, S=S, Ψ=Ψ)
end

function find_Surplus(C::Country2A; 
        reltol=1e-12, findΨ=true, findS=true)
    Σ = find_Σ(C)
    S = findS ? find_S(Σ(C.A)[1:2], C, reltol=reltol) : zeros(2)
    Ψ = findΨ ? find_Ψ(C) : zeros(2)
    return Surplus(Σ=Σ, S=S, Ψ=Ψ)
end